In [1]:
import pandas as pd
import numpy as np
from lightgbm import  Booster, LGBMClassifier
import pickle
import lightgbm as lgb

from sklearn.metrics import accuracy_score, roc_auc_score

print(f"lightgbm version: {lgb.__version__}")

lightgbm version: 4.6.0


In [2]:

# Load the data
test_data = pd.read_csv('data/test.csv')


In [3]:
booster_model = Booster(
    model_file='model.txt'
)

# restore model from pickle file
with open('model.pkl', 'rb') as f:
    orig_model = pickle.load(f)


# create stub classier
model2 = LGBMClassifier()

# populae the stub classifier with the attributes of the original model
for p in dir(orig_model):
    if not p.startswith("__") and not callable(getattr(orig_model, p)):
        print(p, getattr(orig_model, p))
        try:
            setattr(model2, p, getattr(orig_model, p))
        except:
            print(f">>>>>>Error setting {p}")


# Load the booster model into the new classifier
model2._Booster = booster_model
model2.fitted_ = True

X_test = test_data.drop('target', axis=1)
y_test = test_data['target']

# Make predictions on test set
test_predictions = model2.predict(X_test)

# Evaluate the model on test set
test_accuracy = accuracy_score(y_test, test_predictions)
test_roc_auc = roc_auc_score(y_test, test_predictions)

print()
print(f"Test Acc: {test_accuracy:.4f}")
print(f"Test AUC: {test_roc_auc:.4f}")

# Get probability predictions
model2_proba = model2.predict_proba(X_test)

print(model2_proba[:10])

# Save the model as pickle file
with open('model2.pkl', 'wb') as f:
    pickle.dump(model2, f)

_Booster <lightgbm.basic.Booster object at 0xffff194fefd0>
_LGBMClassifier__is_multiclass False
>>>>>>Error setting _LGBMClassifier__is_multiclass
_base_doc 
    Build a gradient boosting model from the training set (X, y).

    Parameters
    ----------
    X : numpy array, pandas DataFrame, H2O DataTable's Frame (deprecated), scipy.sparse, list of lists of int or float of shape = [n_samples, n_features]
        Input feature matrix.
    y : numpy array, pandas DataFrame, pandas Series, list of int or float of shape = [n_samples]
        The target values (class labels in classification, real numbers in regression).
    sample_weight : numpy array, pandas Series, list of int or float of shape = [n_samples] or None, optional (default=None)
        Weights of training data. Weights should be non-negative.
    init_score : numpy array, pandas DataFrame, pandas Series, list of int or float of shape = [n_samples] or shape = [n_samples * n_classes] (for multi-class task) or shape = [n_sampl

In [4]:
# restore model from pickle file
with open('model2.pkl', 'rb') as f:
    model3 = pickle.load(f)


# Make predictions on test set
test_predictions = model3.predict(X_test)

# Evaluate the model on test set
test_accuracy = accuracy_score(y_test, test_predictions)
test_roc_auc = roc_auc_score(y_test, test_predictions)

print()
print(f"Test Acc: {test_accuracy:.4f}")
print(f"Test AUC: {test_roc_auc:.4f}")

# Get probability predictions
model3_proba = model3.predict_proba(X_test)

print(model3_proba[:10])


print(f"probability match: {np.all(model2_proba == model3_proba)}")



Test Acc: 0.9495
Test AUC: 0.9495
[[0.00776503 0.99223497]
 [0.10478753 0.89521247]
 [0.04977558 0.95022442]
 [0.99188038 0.00811962]
 [0.85409501 0.14590499]
 [0.01003624 0.98996376]
 [0.00363611 0.99636389]
 [0.97778551 0.02221449]
 [0.06912174 0.93087826]
 [0.12317774 0.87682226]]
probability match: True
